<a href="https://colab.research.google.com/github/26medias/TF-Face-Angle-Translation/blob/master/Face_Position_Dataset_Builder_One_shot_per_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Face Angle Dataset Generator


## Credits

Face extraction built thanks to https://machinelearningmastery.com/how-to-perform-face-recognition-with-vggface2-convolutional-neural-network-in-keras/

## How this works

1. Download movie trailers
2. Extract the frames from the video files
3. Extract the faces from the images
4. Cluster the faces by actor
5. Build & save the facial landmarks for each face
6. Build the dataset
7. Zip & upload the dataset to Google Storage

## Downloading videos, extracting the frames

We're going to download movie trailers from https://www.davestrailerpage.co.uk/

The frames from the video files will be extracted and saved to file.

## Code setup: Imports & methods

Pip install

In [0]:
!rm -r Faces
!rm -r faces
!rm -r images
!rm -r videos

In [1]:
!pip install git+https://github.com/rcmalli/keras-vggface.git
!pip show keras-vggface
!pip install matplotlib
!pip install mtcnn
!pip install bs4
!pip install selenium

  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-zijhslwx
  Running command git clone -q https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-zijhslwx
  Created wheel for keras-vggface: filename=keras_vggface-0.6-cp36-none-any.whl size=8311 sha256=3b0eb903920e5fc30a244641226742c7b797a0040b121328f6b7138642d6a871
  Stored in directory: /tmp/pip-ephem-wheel-cache-st88y6zn/wheels/36/07/46/06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface
Name: keras-vggface
Version: 0.6
Summary: VGGFace implementation with Keras framework
Home-page: https://github.com/rcmalli/keras-vggface
Author: Refik Can MALLI
Author-email: mallir@itu.edu.tr
License: MIT
Location: /usr/local/lib/python3.6/dist-packages
Requires: h5py, numpy, pillow, keras, six, scipy, pyyaml
Required-by: 


### Code

#### Directory Structure

- Videos
  - [video_filename]
- Faces
  - Clustered
    - [group_id]
      - ...faces
  - Sources
    - [video_filename]
      - ...faces

In [0]:

from google.colab import auth
auth.authenticate_user()

In [40]:
from IPython.display import HTML, display
import time
import requests
import ntpath
import cv2
import math
import os, sys
from matplotlib import pyplot
from PIL import Image
import numpy as np
from numpy import asarray
from scipy.spatial.distance import cosine
from mtcnn.mtcnn import MTCNN
import keras_vggface
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
import glob
import mtcnn
from pathlib import Path
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from scipy.cluster import  hierarchy
from bs4 import BeautifulSoup
from selenium import webdriver

# create the detector, using default weights
print("Creating the detector model")
detector = MTCNN()

# create a vggface model
print("Creating the face embedding model")
embedding_model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')

# The variables
DIR_VIDEOS = "Videos"
DIR_FACES = "Faces"
CAPTURE_FPS  = 23 # We'll extract 1 images per second of video

if not os.path.isdir(DIR_VIDEOS):
  os.mkdir(DIR_VIDEOS, 755);
if not os.path.isdir(DIR_FACES):
  os.mkdir(DIR_FACES, 755);
if not os.path.isdir(DIR_FACES+"/Clustered"):
  os.mkdir(DIR_FACES+"/Clustered", 755);
if not os.path.isdir(DIR_FACES+"/Sources"):
  os.mkdir(DIR_FACES+"/Sources", 755);
if not os.path.isdir(DIR_FACES+"/Previews"):
  os.mkdir(DIR_FACES+"/Previews", 755);

Creating the detector model
Creating the face embedding model


In [0]:


# The methods
# ===========

# Colab progress bar
def progress(value, max=100):
    return HTML('<progress value="{value}" max="{max}" style="width: 50%"> {value}</progress>'.format(value=value, max=max))

# Get the directory of a filename
def getDir(filename):
  p = Path(filename);
  return p.parts[len(p.parts)-2]

# Dowload a video from a url
def downloadFile(url):
  print("Downloading ", url)
  filename = DIR_VIDEOS+"/"+ntpath.basename(url)
  if os.path.exists(filename):
    return filename
  myfile = requests.get(url)
  open(filename, 'wb').write(myfile.content)
  print(filename," downloaded.")
  return filename

def imageFilesToGrid(directory, outputFilename):
  filenames = glob.glob(directory+'/*.jpg')
  print(directory, ": ", len(filenames), " images")
  if len(filenames) < 4:
    return False
  result_figsize_resolution = 10 # 1 = 100px
  
  images_count = len(filenames)
  # Calculate the grid size:
  grid_size = math.ceil(math.sqrt(images_count))
  
  # Create plt plot:
  fig, axes = pyplot.subplots(grid_size, grid_size, figsize=(result_figsize_resolution, result_figsize_resolution))
  
  current_file_number = 0
  for image_filename in filenames:
      x_position = current_file_number % grid_size
      y_position = current_file_number // grid_size
      plt_image = pyplot.imread(image_filename)
      axes[x_position, y_position].imshow(plt_image)
      current_file_number += 1
  pyplot.subplots_adjust(left=0.0, right=1.0, bottom=0.0, top=1.0)
  pyplot.savefig(outputFilename)
  #pyplot.show()

def exportImageGrids(directory, outputDirectory):
  print("Exporting image grids...")
  dirs = os.listdir(directory)
  dirs.sort()
  ndirs = len(dirs)
  for n,dir in enumerate(dirs):
    if dir is not "ALL":
      imageFilesToGrid(directory+"/"+dir, outputDirectory+"/"+dir+".jpg");
    progress(n, ndirs)

# Extract the faces from an image, return an array of numpy faces
def extractFacesFromImage(pixels, required_size=(224, 224), limit=50):
  results = detector.detect_faces(pixels)
  faces = []
  errors = 0
  for i,faceData in enumerate(results):
    if len(faces) > limit:
      break
    x1, y1, width, height = faceData['box']
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    try:
      image = Image.fromarray(face)
      image = image.resize(required_size)
      face_array = asarray(image)
      faces.append(face_array)
      if limit==1:
        return face_array
    except:
      errors+=1
  if limit==1 and len(faces)==0:
    return False
  return faces;


# Export the frames out of a video at a specific fps
def videoToFaces(filename, skipFrame=10, maxFrame=0):
  print("Extracting faces from the video frames...")
  basename = os.path.splitext(ntpath.basename(filename))[0]
  #print("basename:", basename)
  cap = cv2.VideoCapture(filename)
  # Get the video's FPS
  fps = cap.get(cv2.CAP_PROP_FPS)
  nframes = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  print(basename, ": fps: ",fps, " Frames: ", nframes)
  out = display(progress(0, nframes), display_id=True)
  i = 0
  c = 0
  faces = []
  while(cap.isOpened()):
      ret, frame = cap.read()
      if ret == False:
          break
      i+=1
      out.update(progress(i, nframes))
      if maxFrame>0 and i > maxFrame:
        break;
      if (i % skipFrame == 0):
        continue
      frameFaces = extractFacesFromImage(frame)
      for f in frameFaces:
        faces.append(f)
        c+=1
      #cv2.imwrite(DIR_IMAGES+"/"+basename+'/'+str(round((i-1)/fps,2))+'sec.jpg',frame)
  cap.release()
  cv2.destroyAllWindows()
  print(basename, " processed.")
  print(c,"/",i," frames analyzed.")
  print(len(faces), " faces found.")
  return faces


# Show a few images
def showImages(images, width=4):
  fig = pyplot.figure(figsize=(width, math.ceil(len(images)/width)))
  for i in range(len(images)):
      pyplot.subplot(width, math.ceil(len(images)/width), i+1)
      pyplot.imshow(images[i])
      pyplot.axis('off')
  pyplot.savefig('preview.png')
  pyplot.show()

#Save an array of images to files
def saveImages(images, dest, names=False, prefix="", showProgress=True):
  if not os.path.isdir(dest):
    os.mkdir(dest, 755);
  nImages = len(images)
  if showProgress is True:
    print("Saving ",nImages," images to ", dest)
    out = display(progress(0, nImages), display_id=True)
  for n, image in enumerate(images):
    if names is False:
      cv2.imwrite(dest+"/"+prefix+str(n)+'.jpg', image)
    else:
      cv2.imwrite(dest+"/"+prefix+str(names[n])+'.jpg', image)
    if showProgress is True:
      out.update(progress(n, nImages))
      

# Extract faces and calculate face embeddings for a list of photo files
def get_embeddings(faces):
  print("Calculating the embeddings...")
	# convert into an array of samples
  samples = asarray(faces, 'float32')
  # prepare the face for the model, e.g. center pixels
  samples = preprocess_input(samples, version=2)
  # perform prediction
  embeddings = embedding_model.predict(samples)
  return embeddings


# Determine if a candidate face is a match for a known face
def is_match(known_embedding, candidate_embedding, threshold=0.5):
  # calculate distance between embeddings
  score = cosine(known_embedding, candidate_embedding)
  return score >= threshold

# Cluster the faces by cosine distance
def clusterFaces(faces, embeddings):
  groups = [] # Array of dict {faces:[], embeddings: []}
  nFaces = len(faces)
  print("Clustering ",nFaces," faces...")
  out = display(progress(0, nFaces), display_id=True)
  # For each faces
  for n, face in enumerate(faces):
    out.update(progress(n, nFaces))
    if len(groups)==0:
      groups.append({
        "faces":     [face],
        "names":     [n],
        "embeddings":[embeddings[n]]
      })
    else:
      # Not the first face, match it against all the groups, see if the average of cosine distance match an existing face
      scores = [] # array of dict {group: n, embeddings: []}
      for g, group in enumerate(groups):
        groupScores = []
        for embedding in group["embeddings"]:
          groupScores.append(cosine(embedding, embeddings[n]))
        score = np.mean(groupScores)
        scores.append({
            "group": g,
            "score": score
        })
      # Sort the scores for each group by lowest score, check if that score is below the threshold
      scores = sorted(scores, key = lambda i: i["score"], reverse=False)
      if scores[0]["score"] <= 0.5:
        # Add to the existing group the face matches
        groups[scores[0]["group"]]["embeddings"].append(embeddings[n])
        groups[scores[0]["group"]]["faces"].append(face)
        groups[scores[0]["group"]]["names"].append(n)
        #print("[Matched] face #", n, " to group #", scores[0]["group"], "score:", scores[0]["score"])
      else:
        groups.append({
          "faces":     [face],
          "names":     [n],
          "embeddings":[embeddings[n]]
        })
        #print("[New face] face #", n, " / Best score:", scores[0]["score"])
  return groups;

# Cluster all the faces from a remote video
def clusterFacesOnVideo(url):
  print("Processing ", url);
  # Download the video
  videoFilename = downloadFile(url)
  
  # Get the directories name for that video
  dirname      = os.path.splitext(ntpath.basename(videoFilename))[0]
  dirSources   = DIR_FACES+"/Sources/"+dirname
  dirClustered = DIR_FACES+"/Clustered/"+dirname
  dirPreviews  = DIR_FACES+"/Previews/"+dirname
  
  if os.path.exists(dirPreviews):
    # Video already processed, go to the next one
    print("Video already processed. Skipping.")
    return False
  
  # Create the directories
  if not os.path.isdir(dirSources):
    os.mkdir(dirSources, 755);
  if not os.path.isdir(dirClustered):
    os.mkdir(dirClustered, 755);
  if not os.path.isdir(dirPreviews):
    os.mkdir(dirPreviews, 755);
  
  # Find the faces on the video
  faces        = videoToFaces(videoFilename, CAPTURE_FPS)
  nFaces       = len(faces)
  print(nFaces," faces detected")
  
  # Save all the faces
  saveImages(faces, dirSources, prefix=dirname+"_")
  
  # Get the embedding for all the faces
  embeddings    = get_embeddings(faces)
  
  # Cluster the faces using cosine distance
  clusters      = clusterFaces(faces, embeddings)
  nClusters     = len(clusters)
  
  # Export each face group
  print("Saving ",nClusters," face clusters...")
  for n, group in enumerate(clusters):
    saveImages(group["faces"], dirClustered+"/"+str(n), group["names"], showProgress=False)
  
  # Build grids to show each face groups
  exportImageGrids(dirClustered, dirPreviews)


def clusterFacesFromVideos(urls):
  nUrls = len(urls)
  for n,url in enumerate(urls):
    clusterFacesOnVideo(url)

def fetchAllHDVideos(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.content, "html5lib")
  links = soup.find_all('a')
  videos = []
  for tag in links:
    link = tag.get('href', None)
    if link is not None and 'h1080p' in link:
      videos.append(link)
  return videos

## Execute

In [0]:
# Fetch all the HD trailers by webscrapping the webpage
vids = fetchAllHDVideos("https://www.davestrailerpage.co.uk/")

# Cluster the faces from a bunch of videos
clusterFacesFromVideos(vids)

# Save the faces
!tar -zcf faces.tar.gz Faces

# Upload to Cloud Storage
!gcloud config set project deep-learning-files
!gsutil cp  ./faces.tar.gz gs://tf-face-angle-translation/datasets/faces-clustered-large.tar.gz


Processing  http://trailers.apple.com/movies/paramount/terminator-dark-fate/terminator-dark-fate-trailer-2_h1080p.mov
Extracting faces from the video frames...
terminator-dark-fate-trailer-2_h1080p : fps:  23.976023976023978  Frames:  3774
